In [40]:
# -----------------------
# app.ipynb / app.py
# -----------------------
import os
import dash
import pandas as pd
from dash import dcc, html, Input, Output, dash_table
from strava_utils import load_data, create_folium_map, compute_metrics, create_heatmap_figure, weekly_distance_figure, personal_bests_table


# -----------------------
# Load data
# -----------------------
df = load_data('data/nicole_strava.csv')

# -----------------------
# Initialize Dash
# -----------------------
app = dash.Dash(__name__)

# -----------------------
# Layout (Responsive)
# -----------------------
app.layout = html.Div(
    [
        # -----------------------
        # Banner with Image + Title + Filters
        # -----------------------
        html.Div(
            [
                # Athlete picture
                html.Img(
                    src="assets/athlete.png",
                    style={
                        "height": "60px",
                        "width": "60px",
                        "borderRadius": "50%",
                        "marginRight": "15px",
                        "flexShrink": 0
                    }
                ),

                # Title
                html.Div(
                    "Strava Dashboard",
                    style={
                        'color': 'white',
                        'fontSize': '20px',
                        'fontWeight': 'bold',
                        'textAlign': 'center',
                        'marginRight': '20px',
                        'flexGrow': 1
                    }
                ),

                # Filters
                html.Div([
                    # Date Range
                    html.Div([
                        dcc.DatePickerRange(
                            id='date-range',
                            min_date_allowed=df['start_date_local'].min().date(),
                            max_date_allowed=df['start_date_local'].max().date(),
                            start_date=df['start_date_local'].min().date(),
                            end_date=df['start_date_local'].max().date(),
                            style={'fontFamily':'Roboto, sans-serif', 'fontSize':'12px', 'height':'28px'}
                        )
                    ], style={'display':'flex','flexDirection':'column','marginRight':'10px'}),

                    # Sport Type
                    html.Div([
                        dcc.Dropdown(
                            id='sport-type',
                            options=[{'label': s, 'value': s} for s in df['sport_type'].dropna().unique()],
                            value=None,
                            multi=True,
                            placeholder="Select sport type(s)",
                            style={'fontFamily':'Roboto, sans-serif', 'fontSize':'12px', 'minWidth':'150px', 'height':'28px'}
                        )
                    ], style={'display':'flex','flexDirection':'column'})
                ], style={'display':'flex','alignItems':'center','flexWrap':'wrap','gap':'10px'})
            ],
            style={
                'display': 'flex',
                'alignItems': 'center',
                'backgroundColor': '#FC5400',
                'padding': '8px',
                'borderRadius': '8px',
                'marginBottom': '20px',
                'flexWrap': 'wrap',
                'gap': '15px'
            }
        ),

        # -----------------------
        # Two-Column Main Dashboard
        # -----------------------
        html.Div([
            # Left Column
            html.Div([
                # Metrics Box
                html.Div(
                    [
                        html.Div([
                            html.H4("Total KM", style={'fontSize':'14px'}),
                            html.H3(id='total-distance', style={'fontSize':'20px','margin':'3px 0'})
                        ], style={'flex':'1 1 20%','margin':'5px','textAlign':'center','backgroundColor':'#ff4b4b','color':'white','padding':'8px','borderRadius':'8px'}),
                        html.Div([
                            html.H4("Avg Pace", style={'fontSize':'14px'}),
                            html.H3(id='avg-pace', style={'fontSize':'20px','margin':'3px 0'})
                        ], style={'flex':'1 1 20%','margin':'5px','textAlign':'center','backgroundColor':'#ffa500','color':'white','padding':'8px','borderRadius':'8px'}),
                        html.Div([
                            html.H4("Total Elevation", style={'fontSize':'14px'}),
                            html.H3(id='total-elev', style={'fontSize':'20px','margin':'3px 0'})
                        ], style={'flex':'1 1 20%','margin':'5px','textAlign':'center','backgroundColor':'#3b82f6','color':'white','padding':'8px','borderRadius':'8px'}),
                        html.Div([
                            html.H4("Activities", style={'fontSize':'14px'}),
                            html.H3(id='activity-count', style={'fontSize':'20px','margin':'3px 0'})
                        ], style={'flex':'1 1 20%','margin':'5px','textAlign':'center','backgroundColor':'#10b981','color':'white','padding':'8px','borderRadius':'8px'}),
                        html.Div([
                            html.H4("Avg Speed", style={'fontSize':'14px'}),
                            html.H3(id='avg-speed', style={'fontSize':'20px','margin':'3px 0'})
                        ], style={'flex':'1 1 20%','margin':'5px','textAlign':'center','backgroundColor':'#8b5cf6','color':'white','padding':'8px','borderRadius':'8px'}),
                    ], style={'display':'flex','justifyContent':'center','flexWrap':'wrap','gap':'10px'}
                ),

                # Heatmap
                html.H3("Workout Time Preference", style={'textAlign':'center'}),
                dcc.Graph(id='heatmap', style={'width':'100%','height':'400px'}),

                # Personal bests table
                html.H3("Personal Bests", style={'textAlign':'center'}),
                dash_table.DataTable(
                    id='personal-bests-table',
                    page_size=5,
                    style_table={'width':'100%','fontFamily':'Roboto, sans-serif'},
                    style_header={'fontWeight':'bold','backgroundColor':'#f0f0f0'},
                    style_cell={'textAlign':'center'}
                )

            ], style={'flex':'1 1 300px','minWidth':'300px','marginRight':'20px'}),  # Left column

            # Right Column
            html.Div([
                # Map
                html.Iframe(
                    id='folium-map',
                    srcDoc=open('map.html','r').read() if os.path.exists('map.html') else "",
                    width='100%',
                    height='500',
                    style={'border':'none','marginBottom':'20px'}
                ),

                # Weekly distance chart
                html.H3("Total Distance (km) by Week", style={'textAlign':'center'}),
                dcc.Graph(id='weekly-distance', style={'width':'100%','height':'300px'})

            ], style={'flex':'1 1 700px','minWidth':'300px','marginLeft':'20px'})  # Right column
        ], style={'display':'flex','flexWrap':'wrap','gap':'20px','marginBottom':'40px'})
    ],
    style={
        'fontFamily':'Roboto, sans-serif',
        'backgroundColor':'#f9f9f9',
        'color':'#333',
        'padding':'20px'
    }
)



# -----------------------
# Callbacks
# -----------------------
@app.callback(
    Output('total-distance','children'),
    Output('avg-pace','children'),
    Output('total-elev','children'),
    Output('activity-count','children'),
    Output('avg-speed','children'),
    Output('heatmap','figure'),
    Output('folium-map','srcDoc'),
    Output('weekly-distance','figure'),
    Output('personal-bests-table','columns'),
    Output('personal-bests-table','data'),
    Input('date-range','start_date'),
    Input('date-range','end_date'),
    Input('sport-type','value')
)
def update_dashboard(start_date, end_date, sport_types):
    dff = df[(df['start_date_local'].dt.date >= pd.to_datetime(start_date).date()) &
             (df['start_date_local'].dt.date <= pd.to_datetime(end_date).date())]
    if sport_types:
        dff = dff[dff['sport_type'].isin(sport_types)]

    total_distance, avg_pace_str, total_elev, activity_count, avg_speed = compute_metrics(dff)
    heatmap_fig = create_heatmap_figure(dff)
    map_file = create_folium_map(dff)
    with open(map_file,'r') as f:
        map_src = f.read()
    weekly_fig = weekly_distance_figure(dff)
    columns, data = personal_bests_table(dff)

    return total_distance, avg_pace_str, total_elev, activity_count, avg_speed, heatmap_fig, map_src, weekly_fig, columns, data

# -----------------------
# Run App
# -----------------------
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
